In [2]:
#Import libraries
import boto3
import json
import numpy

In [3]:
#Setup AWS services
translate = boto3.client(
    service_name='translate', 
    region_name='us-east-1', 
    use_ssl=True)

bedrock_runtime = boto3.client(
    service_name = 'bedrock-runtime', 
    region_name = 'us-east-1'
)

In [4]:
testPhrase = "Hi, how are you doing today? I'm good."

In [5]:
#Translate to Spanish w/ Translate
translateResultEs = translate.translate_text(
    Text = testPhrase, 
    SourceLanguageCode = "en",
    TargetLanguageCode = "es-MX").get('TranslatedText')

In [6]:
#Translate from Spanish back to English w/ Translate
translateResult = translate.translate_text(
    Text = translateResultEs, 
    SourceLanguageCode = "es-MX",
    TargetLanguageCode = "en").get('TranslatedText')

In [7]:
#Translate to Spanish w/ Jurassic
jurassicPromptEs = "Translate this phrase From English to Spanish \(Mexican Dialect\): \n" +  testPhrase + "\nTranslate as closely as possible."

jurassicBodyEs = json.dumps(
    {"prompt": jurassicPromptEs, 
     "maxTokens": 200,
     "temperature": 0.7,
     "topP": 1,
    }
)

jurassicResponseEs = bedrock_runtime.invoke_model(
    body = jurassicBodyEs, 
	modelId = 'ai21.j2-ultra-v1', 
	accept = 'application/json', 
	contentType = 'application/json'
)

jurassicResponseBodyEs = json.loads(jurassicResponseEs.get('body').read())
jurassicResultEs = jurassicResponseBodyEs.get('completions')[0].get('data').get('text').split(" - ")[0].strip() #using split and strip to pull out the first full string

In [8]:
#Translate from Spanish back to English w/ Jurassic
jurassicPrompt = "Translate this phrase from Spanish \(Mexican Dialect\) to English: " +  jurassicResultEs

jurassicBody = json.dumps(
    {"prompt": jurassicPrompt, 
     "maxTokens": 100,
     "temperature": 0,
     "topP": 1,
    }
)

jurassicResponse = bedrock_runtime.invoke_model(
    body = jurassicBody, 
	modelId = 'ai21.j2-ultra-v1', 
	accept = 'application/json', 
	contentType = 'application/json'
)

jurassicResponseBody = json.loads(jurassicResponse.get('body').read())
jurassicResult = jurassicResponseBody.get('completions')[0].get('data').get('text').split(" - ")[0].strip() #using split and strip to pull out the first full string

In [9]:
#Encode results as JSON to be ingested by Titan Embeddings
originalJson = json.dumps({
    "inputText": testPhrase,
})

translateResultJson = json.dumps({
    "inputText": translateResult,
})

jurassicResultJson = json.dumps({
    "inputText": jurassicResult,
})

In [10]:
#Create embeddings using Titan Embeddings
originalEmbeddingResponse = bedrock_runtime.invoke_model(
    body = originalJson, 
    modelId = 'amazon.titan-embed-text-v1', 
    accept = 'application/json', 
    contentType = 'application/json'
)

originalEmbeddingResponseBody = json.loads(originalEmbeddingResponse['body'].read())
originalEmbedding = originalEmbeddingResponseBody.get('embedding')


translateEmbeddingResponse = bedrock_runtime.invoke_model(
    body = translateResultJson, 
    modelId = 'amazon.titan-embed-text-v1', 
    accept = 'application/json', 
    contentType = 'application/json'
)

translateEmbeddingResponseBody = json.loads(translateEmbeddingResponse['body'].read())
translateEmbedding = translateEmbeddingResponseBody.get('embedding')

jurassicEmbeddingResponse = bedrock_runtime.invoke_model(
    body = jurassicResultJson, 
    modelId = 'amazon.titan-embed-text-v1', 
    accept = 'application/json', 
    contentType = 'application/json'
)

jurassicEmbeddingResponseBody = json.loads(jurassicEmbeddingResponse['body'].read())
jurassicEmbedding = jurassicEmbeddingResponseBody.get('embedding')

In [11]:
#Calculate Euclidean distance between embeddings using NumPy
translateDistance = numpy.linalg.norm (numpy.array(originalEmbedding) - numpy.array(translateEmbedding))
jurassicDistance = numpy.linalg.norm (numpy.array(originalEmbedding) - numpy.array(jurassicEmbedding))

In [12]:
#Print results
print("Original Phrase: " + testPhrase)
print("Translate Result: " + translateResult)
print("Translate Euclidean Distance: " + str(translateDistance))
print("Jurassic Result: " + jurassicResult)
print("Jurassic Euclidean Distance: " + str(jurassicDistance))

Original Phrase: Hi, how are you doing today? I'm good.
Translate Result: Hello, how are you doing today? I'm fine.
Translate Euclidean Distance: 8.033278575547287
Jurassic Result: Hi! How are you today? I'm fine!
Jurassic Euclidean Distance: 9.098566716516826
